In [1]:
from datetime import datetime
import requests

# Funcion para obtener el valor del dolcar CCL al dia de la fecha
def valor_ccl():
    FECHA_INICIO = datetime.now()
    FECHA_FINAL = datetime.now()
    # URL de la API
    url_ccl = f"https://mercados.ambito.com//dolarrava/cl/grafico/{FECHA_INICIO.date()}/{FECHA_FINAL.date()}"
    #url_mep = f'https://mercados.ambito.com//dolarrava/mep/grafico/{FECHA_INICIO.date()}/{FECHA_FINAL.date()}'

    # Realizar una solicitud GET
    response = requests.get(url_ccl)

    # Verificar el estado de la respuesta
    if response.status_code == 200:
        # Parsear el contenido JSON de la respuesta
        data = response.json()
        return data[1][1]
    else:
        print(f"Error: {response.status_code}")

dolar_ccl = valor_ccl()
print(dolar_ccl)

1431.64


In [2]:
import pandas as pd
import yfinance as yf

# Creo el DataFrame con mis acciones, el PPC y la cantidad que tengo
df_tickers = pd.DataFrame({
    'Ticker': ['NVDA.BA', 'GOOGL.BA', 'PFE.BA', 'VIST.BA', 'META.BA', 'COME.BA', 'PAMP.BA', 'MELI.BA', 'GGAL.BA', 'AGRO.BA', 'AMZN.BA'],
    'PPC': [7163.88, 3998.94, 9267.58, 14377.5, 10900, 94.62, 2528, 18687.17, 3035, 59.7, 1925],
    'Cantidad': [21, 17, 6, 4, 9, 728, 14, 6, 7, 280, 35]
})

# Las convierto en lista para poder descargar sus datos y los descargo con yfinance
list_tickers = df_tickers['Ticker'].to_list()
yf_t = yf.download(list_tickers, period='1d', interval='2m')['Adj Close']

# Elijo el ultimo registro
yf_t = yf_t.iloc[-1]

# Creo una columna donde agrego esos precios mapeandolos con el indice de la serie anterior ya que son el mismo
df_tickers['Cierre_ARS'] = df_tickers['Ticker'].map(yf_t).round(2)

# Agrego otra columna donde se ve la ganancia/perdida en ARS y una donde tengo el total invertido
df_tickers['Total invertido ARS'] = (df_tickers['Cierre_ARS'] * df_tickers['Cantidad']).round(2)
df_tickers['Gan/Perd ARS'] = ((df_tickers['Cierre_ARS'] * df_tickers['Cantidad']) - (df_tickers['PPC'] * df_tickers['Cantidad'])).round(2)

# Hago lo mismo pero en USD
df_tickers['Total invertido USD'] = (df_tickers['Cantidad'] * (df_tickers['Cierre_ARS'] / dolar_ccl)).round(2)
df_tickers['Gan/Perd USD'] = (df_tickers['Cantidad'] * (df_tickers['Cierre_ARS'] / dolar_ccl)).round(2) - ((df_tickers['PPC'] / dolar_ccl) * df_tickers['Cantidad']).round(2)
df_tickers

[*********************100%%**********************]  11 of 11 completed


,Ticker,PPC,Cantidad,Cierre_ARS,Total invertido ARS,Gan/Perd ARS,Total invertido USD,Gan/Perd USD
0,NVDA.BA,7163.88,21,7690.00,161490.0,11048.52,112.80,7.72
1,GOOGL.BA,3998.94,17,4560.00,77520.0,9538.02,54.15,6.66
2,PFE.BA,9267.58,6,NaN,NaN,NaN,NaN,NaN
3,VIST.BA,14377.50,4,22900.00,91600.0,34090.00,63.98,23.81
4,META.BA,10900.00,9,29650.00,266850.0,168750.00,186.39,117.87
5,COME.BA,94.62,728,229.25,166894.0,98010.64,116.58,68.46
6,PAMP.BA,2528.00,14,2690.00,37660.0,2268.00,26.31,1.59
7,MELI.BA,18687.17,6,20950.00,125700.0,13576.98,87.80,9.48
8,GGAL.BA,3035.00,7,4110.00,28770.0,7525.00,20.10,5.26
9,AGRO.BA,59.70,280,62.90,17612.0,896.00,12.30,0.62


In [3]:
import plotly.express as px
import plotly.graph_objs as go

# Calculo la ganancia total en USD
total_usd = (df_tickers['Gan/Perd USD'].sum()).round(2)

# Grafico la participación x accion en usd
fig_pie = px.pie(df_tickers, values='Total invertido USD', names='Ticker', title='Participación en USD por acción')
fig_pie.update_traces(textposition='outside', textinfo='percent+label')
fig_pie.add_annotation(text=f'La ganancia neta es de {total_usd} USD', x=-0.3, y=1.1, showarrow=False, font=dict(size=10, color='gray'), xref='paper', yref='paper')
fig_pie.show()

# Ganancia/perdida en dolares de cada accion
fig_bars = go.Figure(data=[
    go.Bar(name='Total invertido USD', x=df_tickers['Ticker'], y=df_tickers['Total invertido USD']),
    go.Bar(name='Gan/Perd USD', x=df_tickers['Ticker'], y=df_tickers['Gan/Perd USD'])
])
# Change the bar mode
fig_bars.update_layout(barmode='group', title='Total invertido y Ganancias en USD por accion')
fig_bars.show()